## License Plate Detection using Fuzzy Join 

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/09-license-plate-fuzzy-join.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

### Connect to EvaDB

In [1]:
%pip install --quiet "evadb[vision,notebook]"
import evadb
cursor = evadb.connect().cursor()

Note: you may need to restart the kernel to use updated packages.


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


### Loading the images to EvaDB for analysis

In [2]:
# Download images
!wget -nc "https://www.dropbox.com/s/770stddqfl0psog/license.zip"
!unzip -n license.zip

cursor.query('DROP TABLE IF EXISTS MyImages;').df()

cursor.load("license/Car*.png", "MyImages", format="image").df()

File 'license.zip' already there; not retrieving.

Archive:  license.zip


,0
0,Number of loaded IMAGE: 7


### License Plate Recognition

In [3]:
cursor.query("""DROP UDF IF EXISTS OCRExtractor;""").df()

cursor.query("""DROP UDF IF EXISTS FuzzDistance;""").df()

cursor.create_udf("OCRExtractor", True, '../evadb/udfs/ocr_extractor.py').df()
cursor.create_udf("FuzzDistance", True, '../evadb/udfs/fuzzy_join.py').df()

06-17-2023 00:48:26 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_udf:0082] UDF OCRExtractor does not exist, therefore cannot be dropped.
06-17-2023 00:48:26 WARNING[drop_object_executor:drop_object_executor.py:_handle_drop_udf:0082] UDF FuzzDistance does not exist, therefore cannot be dropped.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
06-17-2023 00:48:26 ERROR [plan_executor:plan_executor.py:execute_plan:0167] Error creating UDF: Couldn't build proto file into descriptor pool: duplicate file name sentencepiece_model.proto
Traceback (most recent call last):
  File "/nethome/jarulraj3/eva/test_evadb/lib/python3.10/site-packages/evadb/executor/create_udf_executor.py", line 148, in _try_initializing_udf
    udf(**udf_args)
  File "/nethome/jarulraj3/eva/test_evadb/lib/python3.10/site-packages/evadb/udfs/abstract/abstract_udf.py", line 33, in

ExecutorError: Error creating UDF: Couldn't build proto file into descriptor pool: duplicate file name sentencepiece_model.proto

In [ ]:
cursor.query(
    "CREATE TABLE IF NOT EXISTS LicensePlateCSV(id INTEGER UNIQUE, label TEXT(30));"
).df()

In [ ]:
cursor.load("data.csv", "LicensePlateCSV",format="csv").df()

In [ ]:
query = cursor.table("MyImages")
query = query.select("OCRExtractor(data)")
query.df()

In [ ]:
query = cursor.table("LicensePlateCSV")
query = query.select("*")

query.df()

### Run Fuzzy Join to match Detected License Plate against Local License Plate Database (csv)

In [ ]:
cursor.query("""
   SELECT * FROM MyImages 
       JOIN LATERAL OCRExtractor(data) AS T(a) 
       JOIN LicensePlateCSV B 
       ON FuzzDistance(T.a, B.label) > 50;
       """).df()